In [1]:
import pandas as pd
import math


from data_structures.graph import TransportGraph
from algorithms.dijkstra import Dijkstra

# Select city

In [8]:
CITY = 'luxembourg'

# Build transitive clousure graph of walking path

In [9]:
# Download walking connections
walk_connections = pd.read_csv(F'data/{CITY}/network_walk.csv', sep=';')
df_walk_invert = walk_connections.copy()
df_walk_invert = df_walk_invert.rename(columns={'from_stop_I': 'to_stop_I', 'to_stop_I': 'from_stop_I'})
walk_connections = pd.concat((walk_connections, df_walk_invert))

#Additional limit 
walk_connections = walk_connections[walk_connections['d_walk'] < 600]

#Build tg graph
tg_walk = TransportGraph(walk_connections=walk_connections)

In [11]:
NODES = tg_walk.nodes

# Calculate transitive clousure

In [12]:
walk_connections_dict = walk_connections[['from_stop_I', 'to_stop_I', 'd_walk']].to_dict('list')
for node in NODES:
    pathfinding = Dijkstra(graph=tg_walk,
                      start_time=0,
                      start_node=node, 
                      end_node=math.inf)
    path = pathfinding.shortest_path(60 )
    for node_to, duration in pathfinding.candidate_weights.items():
        walk_connections_dict['from_stop_I'].append(node)
        walk_connections_dict['to_stop_I'].append(node_to)
        walk_connections_dict['d_walk'].append(duration)

In [18]:
walk_connections_transitive = pd.DataFrame(walk_connections_dict).drop_duplicates()
walk_connections_transitive = walk_connections_transitive[
            walk_connections_transitive['from_stop_I'] != walk_connections_transitive['to_stop_I']]
walk_connections_transitive

,from_stop_I,to_stop_I,d_walk
0,3,36,464
1,11,1160,455
2,11,450,294
3,11,444,247
4,19,23,385
...,...,...,...
166634,2045,121,7756
166635,2045,2040,8136
166636,2045,2048,8429
166639,2047,2034,847


# Save file

In [19]:
walk_connections_transitive.to_csv(F'data/{CITY}/network_walk_transitive.csv', sep=';', index=False)